In [ ]:
import keras

from keras import layers
from sklearn.manifold import TSNE
from keras.datasets import mnist
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Carregando o banco de dados MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Normalizando os valores do banco de dados
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
print(x_train.shape)
print(x_test.shape)

In [ ]:
# Achatando as imagens (array 2D de 28x28 para array 1D de tamanho 784)
x_train = x_train.reshape((60000, 784))
x_test = x_test.reshape(10000, 784)
print(x_train.shape)
print(x_test.shape)

In [ ]:
plt.imshow(x_train[5].reshape(28,28), cmap="gray")

In [ ]:
# Tamanho da dimensão no espaço latente
encoding_dim = 32
# Input de um array achatado de 784 features (28x28 pixels)
input_img = keras.Input(shape=(784,))

In [ ]:
# Layer conectado ao bottleneck de 32 neurônios
encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# Layer que vai reconstruir a imagem a original
decoded = layers.Dense(784, activation='sigmoid')(encoded)

In [ ]:
# Modelo do autoencoder f(x) = x
autoencoder = keras.Model(input_img, decoded)

autoencoder.summary()

In [ ]:
# Modelo que leva a imagem para a sua versão compacta
encoder = keras.Model(input_img, encoded)
# Input do modelo do encoder
encoded_input = keras.Input(shape=(encoding_dim,))

encoder.summary()

In [ ]:
# Pegando a última layer do autoencoder
decoder_layer = autoencoder.layers[-1]
# Modelo que leva a versão reduzida para a versão "original"
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))

decoder.summary()

In [ ]:
# Treinando o modelo
autoencoder.compile(optimizer="adam", loss='mse')

autoencoder.fit(x_train, x_train,
                epochs=20,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

In [ ]:
# Prevendo os valores de de dimensão reduzida
encoded_imgs = encoder.predict(x_test)
# Prevendo os valores da imagem original
decoded_imgs = decoder.predict(encoded_imgs)

In [ ]:
# Previsão (32 dimensões)
plt.imshow(encoded_imgs[10].reshape(8,4), cmap="gray")

In [ ]:
# Original 
plt.imshow(x_test[10].reshape(28,28), cmap="gray")

In [ ]:
# Previsão (784 dimensões)
plt.imshow(decoded_imgs[10].reshape(28,28), cmap="gray")

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # Plotando original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # Plotando previsão
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
plt.show()

In [ ]:
# Rodando o algoritmo t-sne para redução de dimensionalidade
tsne_test = TSNE(n_components=2).fit_transform(x_test)
tsne_encoded = TSNE(n_components=2).fit_transform(encoded_imgs)

In [ ]:
# Projeção 2D do banco de teste
cmap = plt.get_cmap('rainbow', 10)
plt.scatter(tsne_test[:,0], tsne_test[:,1], c=y_test, edgecolor='black', cmap=cmap)
plt.colorbar(drawedges=True)
plt.show()

In [ ]:
# Projeção 2D do espaço latente
cmap = plt.get_cmap('rainbow', 10)
plt.scatter(tsne_encoded[:,0], tsne_encoded[:,1], c=y_test, edgecolor='black', cmap=cmap)
plt.colorbar(drawedges=True)
plt.show()